In [1]:
import numpy
from numpy.random import randn

from openfermion.transforms._jordan_wigner import *

from openfermion.utils._trotter_exp_to_qgates import *

from openfermion.utils import (commutator, count_qubits, expectation,
                               hermitian_conjugated, normal_ordered,
                               jordan_wigner_sparse, jw_hartree_fock_state,
                               s_squared_operator, sz_operator)

from openfermion.utils._unitary_cc import (uccsd_generator,
                                           uccsd_singlet_generator,
                                           uccsd_singlet_get_packed_amplitudes,
                                           uccsd_singlet_paramsize)

In [34]:
test_orbitals = 4

# single_amplitudes = randn(*(test_orbitals,) * 2)
# double_amplitudes = randn(*(test_orbitals,) * 4)

single_amplitudes = numpy.zeros((test_orbitals, test_orbitals),dtype=complex)
double_amplitudes = numpy.zeros((test_orbitals, test_orbitals,
                                        test_orbitals, test_orbitals),dtype=complex)

single_amplitudes[2][1] = 3.0j
# single_amplitudes[2][1] = -3.0

print('singles')
print(single_amplitudes)
# print('doubles')
# print(double_amplitudes)


singles
[[0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+3.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]]


In [35]:
generator = uccsd_generator(single_amplitudes, double_amplitudes)
qubit_op = jordan_wigner(generator)
# qubit_op *= 1.0j

print('openferion pre trotterized generator')
print(generator)
print('')
print(qubit_op)


for qubit_terms, coefficient in qubit_op.terms.items():
    print('')
    print(coefficient)
    for tensor_term in qubit_terms:
        print(tensor_term[1], tensor_term[0])


openferion pre trotterized generator
-3j [1^ 2] +
3j [2^ 1]

(1.5+0j) [X1 Y2] +
(-1.5+0j) [Y1 X2]

(-1.5+0j)
Y 1
X 2

(1.5+0j)
X 1
Y 2


In [4]:
from forestopenfermion.pyquil_connector import qubitop_to_pyquilpauli
from openfermion.ops import QubitOperator


In [5]:
pauli_sum_representation = qubitop_to_pyquilpauli(qubit_op)
# print(pauli_sum_representation)

In [6]:
from pyquil.quil import Program
from forestopenfermion import exponentiate
from pyquil.gates import *

pyquil_program = exponentiate(qubit_op)
pyquil_program = Program(X(0), X(1), pyquil_program)
# print(pyquil_program)
# print(len(pyquil_program))

[H0  H1  H2  Rx3  cX1-0  cX2-1  cX3-2  Rz3  cX3-2  cX2-1  cX1-0  H0  H1  H2  Rx3  H0  H1  Rx2  H3  cX1-0  cX2-1  cX3-2  Rz3  cX3-2  cX2-1  cX1-0  H0  H1  Rx2  H3  H0  Rx1  cX1-0  Rz1  cX1-0  H0  Rx1  H0  Rx1  H2  H3  cX1-0  cX2-1  cX3-2  Rz3  cX3-2  cX2-1  cX1-0  H0  Rx1  H2  H3  H0  Rx1  Rx2  Rx3  cX1-0  cX2-1  cX3-2  Rz3  cX3-2  cX2-1  cX1-0  H0  Rx1  Rx2  Rx3  H0  Rx1  cX1-0  cX2-1  Rz2  cX2-1  cX1-0  H0  Rx1  H0  Rx1  cX1-0  cX3-1  Rz3  cX3-1  cX1-0  H0  Rx1  H0  Rx2  cX1-0  cX2-1  Rz2  cX2-1  cX1-0  H0  Rx2  H0  Rx2  cX1-0  cX2-1  cX3-2  Rz3  cX3-2  cX2-1  cX1-0  H0  Rx2  H0  Rx3  cX1-0  cX2-1  cX3-2  Rz3  cX3-2  cX2-1  cX1-0  H0  Rx3  H0  Rx3  cX1-0  cX3-1  Rz3  cX3-1  cX1-0  H0  Rx3  H0  Rx2  cX2-0  Rz2  cX2-0  H0  Rx2  H0  Rx3  cX2-0  cX3-2  Rz3  cX3-2  cX2-0  H0  Rx3  Rx0  H1  cX1-0  Rz1  cX1-0  Rx0  H1  Rx0  H1  H2  H3  cX1-0  cX2-1  cX3-2  Rz3  cX3-2  cX2-1  cX1-0  Rx0  H1  H2  H3  Rx0  H1  Rx2  Rx3  cX1-0  cX2-1  cX3-2  Rz3  cX3-2  cX2-1  cX1-0  Rx0  H1  Rx2  Rx3  Rx0  H1  cX1-0  cX2-1  Rz2  cX2-1  cX1-0  Rx0  H1  Rx0  H1  cX1-0  cX3-1  Rz3  cX3-1  cX1-0  Rx0  H1  Rx0  Rx1  H2  Rx3  cX1-0  cX2-1  cX3-2  Rz3  cX3-2  cX2-1  cX1-0  Rx0  Rx1  H2  Rx3  Rx0  Rx1  Rx2  H3  cX1-0  cX2-1  cX3-2  Rz3  cX3-2  cX2-1  cX1-0  Rx0  Rx1  Rx2  H3  Rx0  H2  cX1-0  cX2-1  Rz2  cX2-1  cX1-0  Rx0  H2  Rx0  H2  cX1-0  cX2-1  cX3-2  Rz3  cX3-2  cX2-1  cX1-0  Rx0  H2  Rx0  H3  cX1-0  cX2-1  cX3-2  Rz3  cX3-2  cX2-1  cX1-0  Rx0  H3  Rx0  H3  cX1-0  cX3-1  Rz3  cX3-1  cX1-0  Rx0  H3  Rx0  H2  cX2-0  Rz2  cX2-0  Rx0  H2  Rx0  H3  cX2-0  cX3-2  Rz3  cX3-2  cX2-0  Rx0  H3  H1  Rx2  cX1-0  cX2-1  Rz2  cX2-1  cX1-0  H1  Rx2  H1  Rx3  cX1-0  cX2-1  cX3-2  Rz3  cX3-2  cX2-1  cX1-0  H1  Rx3  Rx1  H2  cX1-0  cX2-1  Rz2  cX2-1  cX1-0  Rx1  H2  Rx1  H3  cX1-0  cX2-1  cX3-2  Rz3  cX3-2  cX2-1  cX1-0  Rx1  H3  H2  Rx3  cX2-0  cX3-2  Rz3  cX3-2  cX2-0  H2  Rx3  Rx2  H3  cX2-0  cX3-2  Rz3  cX3-2  cX2-0  Rx2  H3  H1  Rx2  cX2-1  Rz2  cX2-1  H1  Rx2  H1  Rx2  cX2-1  cX3-2  Rz3  cX3-2  cX2-1  H1  Rx2  H1  Rx3  cX2-1  cX3-2  Rz3  cX3-2  cX2-1  H1  Rx3  H1  Rx3  cX3-1  Rz3  cX3-1  H1  Rx3  Rx1  H2  cX2-1  Rz2  cX2-1  Rx1  H2  Rx1  H2  cX2-1  cX3-2  Rz3  cX3-2  cX2-1  Rx1  H2  Rx1  H3  cX2-1  cX3-2  Rz3  cX3-2  cX2-1  Rx1  H3  Rx1  H3  cX3-1  Rz3  cX3-1  Rx1  H3  H2  Rx3  cX2-1  cX3-2  Rz3  cX3-2  cX2-1  H2  Rx3  Rx2  H3 # cX2-1  cX3-2  Rz3  cX3-2  cX2-1  Rx2  H3  H2  Rx3  cX3-2  Rz3  cX3-2  H2  Rx3  Rx2  H3  cX3-2  Rz3  cX3-2  Rx2  H3]

# The correct answer

In [7]:
from pyquil.api import WavefunctionSimulator
from pyquil import Program


print('')
wfn = WavefunctionSimulator().wavefunction(pyquil_program)
print(wfn)


(-0.5651687486+0j)|0011> + (0.223169612+0j)|0101> + (-0.0505823243+0j)|0110> + (-0.7514238465+0j)|1001> + (-0.2042297359+0j)|1010> + (0.1478967756+0j)|1100>


In [8]:
['(-0.000000 +0.000000 i) |0000>',
 '(0.000000 +0.000000 i) |1000>',
 '(0.000000 -0.000000 i) |0100>',
 '(-0.089068 +0.000000 i) |1100>',
 '(0.000000 -0.000000 i) |0010>',
 '(0.555595 +0.000000 i) |1010>',
 '(0.509890 +0.000000 i) |0110>',
 '(0.000000 -0.000000 i) |1110>',
 '(0.000000 -0.000000 i) |0001>',
 '(-0.090056 +0.000000 i) |1001>',
 '(0.643190 +0.000000 i) |0101>',
 '(0.000000 -0.000000 i) |1101>',
 '(-0.039881 +0.000000 i) |0011>',
 '(0.000000 -0.000000 i) |1011>',
 '(0.000000 -0.000000 i) |0111>',
 '(-0.000000 +0.000000 i) |1111>']

['(-0.000000 +0.000000 i) |0000>',
 '(0.000000 +0.000000 i) |1000>',
 '(0.000000 -0.000000 i) |0100>',
 '(-0.089068 +0.000000 i) |1100>',
 '(0.000000 -0.000000 i) |0010>',
 '(0.555595 +0.000000 i) |1010>',
 '(0.509890 +0.000000 i) |0110>',
 '(0.000000 -0.000000 i) |1110>',
 '(0.000000 -0.000000 i) |0001>',
 '(-0.090056 +0.000000 i) |1001>',
 '(0.643190 +0.000000 i) |0101>',
 '(0.000000 -0.000000 i) |1101>',
 '(-0.039881 +0.000000 i) |0011>',
 '(0.000000 -0.000000 i) |1011>',
 '(0.000000 -0.000000 i) |0111>',
 '(-0.000000 +0.000000 i) |1111>']

# Now begin importing trial circuit to Qforte

In [9]:
import qforte

In [10]:
#get from openferion
qforte_generator = qforte.build_from_openfermion(qubit_op)
#qforte.smart_print(qforte_generator)

# Now try to exponentiate the full operator 

In [11]:
expn_generator = qforte.trotterization.trotterize(qforte_generator)
#qforte.smart_print(expn)
#expn.str()

In [12]:
# Set HF state
qc2 = qforte.QuantumComputer(4)
qc2.apply_gate(qforte.make_gate('X',0,0))
qc2.apply_gate(qforte.make_gate('X',1,1))
#qc2.str()

In [13]:
qc2.apply_circuit(expn_generator)
qc2.str()

['(-0.000000 +0.000000 i) |0000>',
 '(0.000000 +0.000000 i) |1000>',
 '(0.000000 +0.000000 i) |0100>',
 '(-0.565169 +0.000000 i) |1100>',
 '(0.000000 -0.000000 i) |0010>',
 '(0.223170 +0.000000 i) |1010>',
 '(-0.050582 +0.000000 i) |0110>',
 '(0.000000 +0.000000 i) |1110>',
 '(0.000000 +0.000000 i) |0001>',
 '(-0.751424 +0.000000 i) |1001>',
 '(-0.204230 +0.000000 i) |0101>',
 '(0.000000 +0.000000 i) |1101>',
 '(0.147897 +0.000000 i) |0011>',
 '(0.000000 -0.000000 i) |1011>',
 '(0.000000 -0.000000 i) |0111>',
 '(-0.000000 +0.000000 i) |1111>']

# Now set up for variational optemization via VQE

In [14]:
# import to get hamiltonian from psi 
import unittest
import qforte
from openfermion.hamiltonians import MolecularData
from openfermion.transforms import get_fermion_operator, jordan_wigner
from openfermionpsi4 import run_psi4
import numpy

In [15]:
# Set molecule parameters.
basis = 'sto-3g'
multiplicity = 1
bond_length_interval = 0.75
n_points = 1

# Set calculation parameters.
run_scf = 1
run_mp2 = 0
run_cisd = 0
run_ccsd = 1
run_fci = 1
delete_input = True
delete_output = True

# Generate molecule at different bond lengths.
hf_energies = []
fci_energies = []
bond_lengths = []
for point in range(1, n_points + 1):
    bond_length = bond_length_interval * float(point)
    bond_lengths += [bond_length]
    geometry = [('H', (0., 0., 0.)), ('H', (0., 0., bond_length))]
    molecule = MolecularData(
        geometry, basis, multiplicity,
        description=str(round(bond_length, 2)))
    
    # Run Psi4.
    molecule = run_psi4(molecule,
                        run_scf=run_scf,
                        run_mp2=run_mp2,
                        run_cisd=run_cisd,
                        run_ccsd=run_ccsd,
                        run_fci=run_fci)
    
print('Hartree-Fock energy of {} Hartree.'.format(molecule.hf_energy))
print('Hartree-Fock energy of {} Hartree.'.format(molecule.ccsd_energy))
print('Hartree-Fock energy of {} Hartree.'.format(molecule.fci_energy))

Hartree-Fock energy of -1.116151448914139 Hartree.
Hartree-Fock energy of -1.1371170673687787 Hartree.
Hartree-Fock energy of -1.1371170673370452 Hartree.


In [16]:
#populate data and integrals
molecule.load()

# Get the Hamiltonian in an active space.
active_space_start = 0
active_space_stop = 2

molecular_hamiltonian = molecule.get_molecular_hamiltonian(
    occupied_indices=range(active_space_start),
    active_indices=range(active_space_start, active_space_stop))

# Map operator to fermions and qubits.
fermion_hamiltonian = get_fermion_operator(molecular_hamiltonian)
qubit_hamiltonian = jordan_wigner(fermion_hamiltonian)
#qubit_hamiltonian.compress()
# print('The Jordan-Wigner Molecular Hamiltonian in canonical basis:\n{}'.format(qubit_hamiltonian))

# print('\nBuild Qforte Hamiltonian and generator to calculate the expectation value')
qforte_hamiltonian = qforte.build_from_openfermion(qubit_hamiltonian)
# print('\nThe Molecular Hamiltonian in canonical basis:')
# qforte.smart_print(qforte_hamiltonian)

# Set up VQE object

In [17]:
myVQE = qforte.VQE(4, 2, expn_generator, qforte_hamiltonian, 5000) 

In [18]:
circ = qforte.QuantumCircuit()
Exper2 = qforte.Experiment(4, 2, expn_generator, qforte_hamiltonian, 5000)
params2 = [0.0] * 44 
print(Exper2.experimental_avg(params2))

AttributeError: 'Experiment' object has no attribute 'RHF_ref'

# Perform the porcedure 

In [ ]:
optemized = myVQE.do_vqe()
print(optemized.fun)


In [ ]:
print(optemized.x)